In [13]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [60]:
import pandas as pd
autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][0])
senha = str(autentica['senha'][0])

# display(autentica)

pedidos = pd.read_excel('consolidacao/pedidos/CAFI_pedidos.xlsx')
pedidos = pedidos[pedidos['acao'].str.contains('pedido no suap')]

# separando elementos de despesa
pedidos_ed = pedidos['ed'].drop_duplicates()
pedidos_ed.index = [x for x in range(0, len(pedidos_ed))]
print(pedidos_ed)

pedidos = pedidos[pedidos['ed'].str.contains(str(pedidos_ed[12]))]


# reindexando os pedidos
pedidos.index = [x for x in range(1, len(pedidos)+1)]
pedidos['i'] = [x for x in range(1, len(pedidos)+1)]
# pedidos = pedidos.iloc[0:6]
display(pedidos)
display(pedidos.info())
qtd = pedidos[['i', 'diferenca']]
dict_qtd = qtd.to_dict('records')
ped = pedidos[['i', 'codigo']]
dict_ped = ped.to_dict('records')

indisponivel = "indisponivel"

0                   14 - MATERIAL EDUCATIVO E ESPORTIVO
1                           16 - MATERIAL DE EXPEDIENTE
2               17 - MATERIAL DE PROCESSAMENTO DE DADOS
3         19 - MATERIAL DE ACONDICIONAMENTO E EMBALAGEM
4                       21 - MATERIAL DE COPA E COZINHA
5      22 - MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO
6     24 - MATERIAL P/ MANUT.DE BENS IMOVEIS/INSTALA...
7            25 - MATERIAL P/ MANUTENCAO DE BENS MOVEIS
8                   26 - MATERIAL ELETRICO E ELETRONICO
9                 28 - MATERIAL DE PROTECAO E SEGURANCA
10                           35 - MATERIAL LABORATORIAL
11                                     42 - FERRAMENTAS
12         44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS
Name: ed, dtype: object


,codigo,ed,descricao_suap,qtd_suap,qtd_lev,diferenca,acao,i
1,30.44.67,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,"Fita adesiva, material propileno, 20mm x 30m, ...",11,10,1,pedido no suap,1


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 1 to 1
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   codigo          1 non-null      object
 1   ed              1 non-null      object
 2   descricao_suap  1 non-null      object
 3   qtd_suap        1 non-null      int64 
 4   qtd_lev         1 non-null      int64 
 5   diferenca       1 non-null      int64 
 6   acao            1 non-null      object
 7   i               1 non-null      int64 
dtypes: int64(4), object(4)
memory usage: 72.0+ bytes


None

In [15]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
import time

def abrir_suap():
    
    global nav
    nav = webdriver.Chrome()
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    # inserindo usuário
    nav.find_element_by_xpath('//*[@id="id_username"]').click()
    nav.find_element_by_xpath('//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element_by_xpath('//*[@id="id_password"]').click()
    nav.find_element_by_xpath('//*[@id="id_password"]').send_keys(senha)

    nav.find_element_by_xpath('//*[@id="ok"]').click()

In [16]:
df_elemento_despesa = pd.DataFrame()
df_temp = pd.DataFrame(columns=('value', 'ed'))
df_temp.loc[0] = ['14', 'esportes']
# display(df_temp)

opcoes = []
valores = []

def varrer_ed():
#     abrir_suap()
    nav.get("https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/")
        
    valor_ed = nav.find_element_by_id('id_elemento_despesa')
    options = [x for x in valor_ed.find_elements_by_tag_name('option')]
    
    for i in options:
        # print(i.get_attribute('value')+" "+i.text)
        opcoes.append(i.text)
        valores.append(i.get_attribute('value'))
#         print(f'{i.text}-{i.get_attribute("value")}')

    df_elemento_despesa['value'] = valores
    df_elemento_despesa['ed']= opcoes
    
#     display(df_elemento_despesa)

In [17]:
def get_qtd():
    
    abrir_suap()
    
    df_tabela = pd.DataFrame()
    tabela = []
    codigos = []
    descricoes = []
    quantidades = []
    valores = []
    elemento = []
    
    try:
        pag = 1
        saida = True
        while saida:
                
            nav.get(f'https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/?page={pag}&material=&elemento_despesa=&estoque_tipo=emcampus&unidade_medida=')
            celula = nav.find_element_by_tag_name('td')
                
            if celula:
                lista = nav.find_element_by_tag_name('table')
                td = lista.find_elements_by_tag_name('td')
                
                for i in td:
                    tabela.append(i.text)
                
                pag += 1
            else:
                saida = False
                
    except NoSuchElementException:
        print('acabaram as páginas')
        
    for cod in range(0, len(tabela), 7):
        codigos.append(tabela[cod])

    for des in range(1, len(tabela), 7):
        descricoes.append(tabela[des])

    for qtd in range(4, len(tabela), 7):
        quantidades.append(tabela[qtd])
    
    for val in range(5, len(tabela), 7):
        valores.append(tabela[val])
        
    for ed in range(3, len(tabela), 7):
        elemento.append(tabela[ed])

    df_tabela['codigo'] = codigos
    df_tabela['descricao'] = descricoes
    
    df_tabela['qtd'] = quantidades
    df_tabela['qtd'] = pd.to_numeric(df_tabela['qtd'])
    
    df_tabela['vlr_total'] = valores
    df_tabela['vlr_total'] = df_tabela['vlr_total'].str[2:]
    df_tabela['vlr_total'] = pd.to_numeric(df_tabela['vlr_total'])
    
    df_tabela['vlr_unt'] = df_tabela['vlr_total'] / df_tabela['qtd']
    df_tabela['vlr_unt'] = df_tabela['vlr_unt'].round(2)
    
    df_tabela['ed'] = elemento
    
    df_tabela.to_excel('relatorio/relatorio.xlsx', index=False)
    display(df_tabela)
    
    nav.quit()


In [31]:
def set_pedidos():
    abrir_suap()

    nav.get('https://suap.ifpe.edu.br/almoxarifado/form_requisicao_pessoa_pedido/')

    # preenchendo campo do requisitante
    nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/fieldset/div[2]/input[1]').send_keys('Andrea Dacal')
    time.sleep(3)
    nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/fieldset/div[2]/input[1]').send_keys(Keys.DOWN)
    nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/fieldset/div[2]/input[1]').send_keys(Keys.ENTER)

    # preenchento campo do pedido do material

    for i in dict_ped:
        nav.find_element_by_xpath(f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[{i["i"]}]/input[1]').send_keys(i['codigo'])
        time.sleep(3)
        nav.find_element_by_xpath(f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[{i["i"]}]/input[1]').send_keys(Keys.DOWN)
        nav.find_element_by_xpath(f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[{i["i"]}]/input[1]').send_keys(Keys.ENTER)
        time.sleep(2)
        nav.find_element_by_name('addItem').click()

    for i in dict_qtd:    
        nav.find_element_by_xpath(f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[{i["i"]}]/input[3]').send_keys(i['diferenca'])

In [61]:
set_pedidos()